In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import scipy.stats as stats
import gmaps
import os
import json
import math

from pprint import pprint


# Import API key
#from config import g_key

In [3]:
#read in income data
home_value_file = "Neighborhood_Zhvi_AllHomes.csv"
home_value_df = pd.read_csv(home_value_file)
home_value_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16134 entries, 0 to 16133
Columns: 300 entries, RegionID to 2020-03-31
dtypes: float64(291), int64(2), object(7)
memory usage: 36.9+ MB


In [8]:
#filter to only atlanta data and create new dataframe
atl_home_value_df = home_value_df.loc[home_value_df.City == "Atlanta"].copy()
atl_home_value_df.dropna()
atl_home_value_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31
701,269381,733,Midtown,Neighborhood,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,125032.0,...,322186.0,321732.0,321479.0,321212.0,321213.0,321620.0,323318.0,325124.0,327471.0,330131.0
873,269306,909,Downtown,Neighborhood,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,76496.0,...,242416.0,242250.0,242301.0,242484.0,242322.0,242545.0,243084.0,244109.0,245747.0,248704.0
1245,269397,1290,Old Fourth Ward,Neighborhood,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,113476.0,...,328601.0,328846.0,328809.0,328789.0,328527.0,329002.0,330664.0,332188.0,334206.0,336817.0
1773,269391,1834,North Buckhead,Neighborhood,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,219535.0,...,535498.0,534661.0,533890.0,533569.0,533116.0,532864.0,533406.0,533620.0,535212.0,536955.0
1782,274592,1843,Morningside - Lenox Park,Neighborhood,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,248884.0,...,819594.0,820440.0,820473.0,820941.0,822338.0,822987.0,823917.0,826341.0,830985.0,833870.0


In [9]:
#get rid of years we dont have crime data for
atl_home_value_df.drop(atl_home_value_df.iloc[:, 9:165], inplace = True, axis = 1) 
atl_home_value_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2009-01-31,...,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31
701,269381,733,Midtown,Neighborhood,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,250118.0,...,322186.0,321732.0,321479.0,321212.0,321213.0,321620.0,323318.0,325124.0,327471.0,330131.0
873,269306,909,Downtown,Neighborhood,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,167964.0,...,242416.0,242250.0,242301.0,242484.0,242322.0,242545.0,243084.0,244109.0,245747.0,248704.0
1245,269397,1290,Old Fourth Ward,Neighborhood,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,220787.0,...,328601.0,328846.0,328809.0,328789.0,328527.0,329002.0,330664.0,332188.0,334206.0,336817.0
1773,269391,1834,North Buckhead,Neighborhood,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,427563.0,...,535498.0,534661.0,533890.0,533569.0,533116.0,532864.0,533406.0,533620.0,535212.0,536955.0
1782,274592,1843,Morningside - Lenox Park,Neighborhood,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,542329.0,...,819594.0,820440.0,820473.0,820941.0,822338.0,822987.0,823917.0,826341.0,830985.0,833870.0


In [4]:
#get a list of atlanta neighborhoods with income data
home_value_neighborhoods = atl_home_value_df.RegionName.unique()

In [5]:
#location = "Atlanta_crimedata/COBRA-YTD2017.csv"
crime_file = "COBRA-2009-2019.csv"
crimes_df = pd.read_csv(crime_file)
crimes_df.info()

C:\Users\NIC\Anaconda3\envs\pythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342914 entries, 0 to 342913
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Report Number            342914 non-null  int64  
 1   Report Date              342914 non-null  object 
 2   Occur Date               342914 non-null  object 
 3   Occur Time               342914 non-null  object 
 4   Possible Date            342896 non-null  object 
 5   Possible Time            342895 non-null  float64
 6   Beat                     342890 non-null  float64
 7   Apartment Office Prefix  10094 non-null   object 
 8   Apartment Number         68274 non-null   object 
 9   Location                 342912 non-null  object 
 10  Shift Occurence          342914 non-null  object 
 11  Location Type            333698 non-null  object 
 12  UCR Literal              342914 non-null  object 
 13  UCR #                    342914 non-null  int64  
 14  IBR 

In [6]:
len(crimes_df["Report Number"].unique())

342609

In [7]:
#removing unnecessary columns
crimes1_df = crimes_df.drop(["Report Date","Possible Date","Possible Time",
                                   "Beat","Apartment Office Prefix","Apartment Number",
                                   "Shift Occurence","Location Type","IBR Code","NPU"],axis=1)

crimes1_df

,Report Number,Occur Date,Occur Time,Location,UCR Literal,UCR #,Neighborhood,Latitude,Longitude
0,90010930,1/1/09,1145,2841 GREENBRIAR PKWY,LARCENY-NON VEHICLE,630,Greenbriar,33.68845,-84.49328
1,90011083,1/1/09,1330,12 BROAD ST SW,LARCENY-NON VEHICLE,630,Downtown,33.75320,-84.39201
2,90011208,1/1/09,1500,3500 MARTIN L KING JR DR SW,LARCENY-NON VEHICLE,630,Adamsville,33.75735,-84.50282
3,90011218,1/1/09,1450,3393 PEACHTREE RD NE,LARCENY-NON VEHICLE,630,Lenox,33.84676,-84.36212
4,90011289,1/1/09,1600,2841 GREENBRIAR PKWY SW,LARCENY-NON VEHICLE,630,Greenbriar,33.68677,-84.49773
...,...,...,...,...,...,...,...,...,...
342909,193652089,12/31/19,2030,1385 SHARON ST NW,AGG ASSAULT,420,Hunter Hills,33.75486,-84.43287
342910,193650336,12/31/19,432,262 PHARR RD NE,AGG ASSAULT,410,Buckhead Village,33.83732,-84.37860
342911,193650603,12/31/19,920,689 CASCADE AVE SW,AGG ASSAULT,410,Westview,33.73636,-84.43680
342912,193651760,12/31/19,1853,763 CASCADE AVE SW,AGG ASSAULT,410,Westview,33.73483,-84.43750


In [8]:
crimes1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342914 entries, 0 to 342913
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Report Number  342914 non-null  int64  
 1   Occur Date     342914 non-null  object 
 2   Occur Time     342914 non-null  object 
 3   Location       342912 non-null  object 
 4   UCR Literal    342914 non-null  object 
 5   UCR #          342914 non-null  int64  
 6   Neighborhood   330551 non-null  object 
 7   Latitude       342914 non-null  float64
 8   Longitude      342914 non-null  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 23.5+ MB


In [14]:
#deleting the rows with null values
cleaned_crimes_df=crimes1_df.dropna(axis=0,how='any')
cleaned_crimes_df.info()
#get a list of neighborhoods with crime data
crimes_neighborhoods = cleaned_crimes_df.Neighborhood.unique()

In [30]:
#compare neighborhood lists and create list of neighborhoods in both data sets
home_value_neighborhoods.sort()
crimes_neighborhoods.sort()
print(len(home_value_neighborhoods))
print(len(crimes_neighborhoods))
neighborhoods = np.intersect1d(home_value_neighborhoods, crimes_neighborhoods)


180
243


In [20]:
# #Dictionary of Median income per neighborhood and population
# income_dict = {"Paces": [166.6, 2.69],
#               "Brookwood Hills": [148.6, 2.7],
#               "North Buckhead": [128.3, 8.34],
#               "Morningside/Lenox Park": [127.4, 8.02],
#               "Lake Claire": [111.9, 2.7],
#               "Virginia Highland": [93.7, 8.08],
#               "Oakhurst": [92.8, 3.94],
#                "Candler Park": [92.4, 3.4],
#                "Grant Park": [90.9, 6.56],
#                "Ormewood Park": [87.2, 4.19],
#                "Inman Park": [84.3, 4.22],
#                "Peachtree Hills": [83.1, 2.66],
#                "Pine Hills": [80.9, 8.3],
#                "Garden Hills": [79.1, 3.58],
#                "Midtown": [77.6, 17.06],
#                "Bolton": [76.4, 3.92],
#                "Peachtree Heights West": [74.1, 4.87],
#                "Knight Park/Howell Station": [69.7, 2.64],
#                "Kirkwood": [69.1, 5.78],
#                "Underwood Hills": [67.9, 3.17],
#                "East Atlanta": [64.8, 4.96],
#                "Lindridge/Martin Manor": [58.7, 4.21],
#                "Lindbergh/Morosgo": [57.5, 4.6],
#                "Home Park": [52.0, 4.75],
#                "Downtown": [45.1, 13.36],
#                "Chosewood Park": [41.7, 3.99],
#                "Old Fourth Ward": [38.9, 10.5],
#                "Edgewood": [38.6, 3.98],
#                "Westview": [34.9, 3.14],
#                "Venetian Hills": [31.3, 3.71],
#                "Glenrose Heights": [29.6, 4.07],
#                "Georgia Tech": [28.8, 6.53],
#                "Grove Park": [28.2, 4.65],
#                "Center Hill": [27.7, 3.01],
#                "West End": [27.6, 4.34],
#                "Atlanta University Center": [27.3, 5.91],
#                "Collier Heights": [26.6, 5.68],
#                "Peoplestown": [26.4, 2.66],
#                "Vine City": [25.6, 2.82],
#                "Sylvan Hills": [25.4, 4.63],
#                "Pittsburgh": [22.3, 3.47],
#                "Greenbriar": [21.8, 3.63],
#                "Fairburn Mays": [21.7, 3.42],
#                "English Avenue": [21.4, 3.33],
#                "Oakland City": [20.4, 3.48],
#                "Hammond Park": [20.1, 2.7],
#                "Mechanicsville": [18.3, 3.79],
#                "Campbellton Road": [14.6, 5.08],
#                "Thomasville Heights": [12.2, 2.7]
#               }

In [11]:
#crime dictionary
crime_dict = {"HOMICIDE": 110,
            "MANSLAUGHTER": 120,
            "ROBBERY-PEDESTRIAN": 311,
            "ROBBERY-COMMERCIAL": 312,
            "ROBBERY-COMMERCIAL": 313,
            "ROBBERY-COMMERCIAL": 314,
            "ROBBERY-RESIDENCE": 315,
            "ROBBERY-COMMERCIAL": 316,
            "ROBBERY-PEDESTRIAN": 317,
            "ROBBERY-PEDESTRIAN": 321,
            "ROBBERY-COMMERCIAL": 322,
            "ROBBERY-COMMERCIAL": 323,
            "ROBBERY-COMMERCIAL": 324,
            "ROBBERY-RESIDENCE": 325,
            "ROBBERY-COMMERCIAL": 326,
            "ROBBERY-PEDESTRIAN": 327,
            "ROBBERY-PEDESTRIAN": 331,
            "ROBBERY-COMMERCIAL": 332,
            "ROBBERY-COMMERCIAL": 333,
            "ROBBERY-COMMERCIAL": 334,
            "ROBBERY-RESIDENCE": 335,
            "ROBBERY-COMMERCIAL": 336,
            "ROBBERY-PEDESTRIAN": 337,
            "ROBBERY-PEDESTRIAN": 341,
            "ROBBERY-COMMERCIAL": 342,
            "ROBBERY-COMMERCIAL": 343,
            "ROBBERY-COMMERCIAL": 344,
            "ROBBERY-RESIDENCE": 345,
            "ROBBERY-COMMERCIAL": 346,
            "ROBBERY-PEDESTRIAN": 347,
            "AGG ASSAULT": 410,
            "AGG ASSAULT": 420,
            "AGG ASSAULT": 430,
            "AGG ASSAULT": 440,
            "BURGLARY-RESIDENCE": 511,
            "BURGLARY-NONRES": 512,
            "BURGLARY-RESIDENCE": 521,
            "BURGLARY-NONRES": 522,
            "BURGLARY-RESIDENCE": 531,
            "BURGLARY-NONRES": 532,
            "LARCENY-NON VEHICLE": 610,
            "LARCENY-NON VEHICLE": 620,
            "LARCENY-NON VEHICLE": 630,
            "LARCENY-FROM VEHICLE": 640,
            "LARCENY-FROM VEHICLE": 650,
            "LARCENY-NON VEHICLE": 660,
            "LARCENY-NON VEHICLE": 670,
            "LARCENY-NON VEHICLE": 680,
            "LARCENY-NON VEHICLE": 690,
            "AUTO THEFT": 710,
            "AUTO THEFT": 720,
            "AUTO THEFT": 730
}

In [12]:
#Median income dataframe
# income_df = pd.DataFrame(list(income_dict.items()), columns = ["Neighborhood", "Median Income (K)"])
# income_df.head()
income_df = pd.DataFrame.from_dict(income_dict, orient='index', columns=["Median Income (K)", "Population (K)"]).reset_index()
income_df.rename(columns = {"index":"Neighborhood"}, inplace = True)


NameError: name 'income_dict' is not defined

In [ ]:
#Merge all data into one dataframe
merged_df = pd.merge(cleaned_crimes_df, income_df, how = "outer", on="Neighborhood")
final_df = merged_df.dropna(axis = 0, how = "any")
final_df.head()

In [ ]:
#Make a dataframe with crimes per capita, for each neighborhood

hood_group = final_df.groupby("Neighborhood").count()
new_df = hood_group["Report Number"].copy()
pop_df = pd.merge(new_df, income_df, how = "outer", on = "Neighborhood")
pop_df.rename(columns={"Report Number":"Total Crimes"}, inplace = True)
pop_df["Crime Per Capita"] = pop_df["Total Crimes"] / (pop_df["Population (K)"]*1000)
pop_df.dropna(how = "any", axis = 0, inplace = True)
pop_df.head()

In [ ]:
#Split neighborhoods into different quartiles based on median income
quartiles = pop_df["Median Income (K)"].quantile([.25, .5, .75])
#upper quartile
up_quart = quartiles[.75]
#median
mid_quart = quartiles[.5]
#lower quartile
low_quart = quartiles[.25]
#inner quartile range
iqr = up_quart - low_quart
#top of whisker
box_max = up_quart + (1.5*iqr)
#bottom of whisker
box_min = low_quart - (1.5*iqr)
#all neighborhoods whose median income falls between bottom whisker and bottom of the box
bottom = pop_df.loc[(pop_df["Median Income (K)"] < low_quart) & (pop_df["Median Income (K)"] > box_min)]
#all neighborhhods whose median income falls between bottom of the box and the median
mid_bottom = pop_df.loc[(pop_df["Median Income (K)"] < mid_quart) & (pop_df["Median Income (K)"] > low_quart)]
#all neighborhoods whose median income falls between median and top of the box
mid_top = pop_df.loc[(pop_df["Median Income (K)"] < up_quart) & (pop_df["Median Income (K)"] > mid_quart)]
#all neighborhoods whose median income falls betweentop of the box and top whisker
top = pop_df.loc[(final_df["Median Income (K)"] < box_max) & (pop_df["Median Income (K)"] > up_quart)]
#count up the crimes in each income range
bot_count = bottom["Crime Per Capita"].mean()
midbot_count = mid_bottom["Crime Per Capita"].mean()
midtop_count = mid_top["Crime Per Capita"].mean()
top_count = top["Crime Per Capita"].mean()
#lists for x and y values
crime_counts = [bot_count, midbot_count, midtop_count, top_count]
crime_quarts = ["Bottom", "Mid-Bottom", "Mid-Top", "Top"]
#bar chart showing number of crime vs neighborhood income quartiles
plt.bar(crime_quarts, crime_counts, color = "b", align = "center")
plt.xlabel("Neighborhood Income Quartiles")
plt.ylabel("Crimes Per Capita")

In [ ]:
#Scatter plot ith crime per capita and median income by neighborhood
scat_x = pop_df["Median Income (K)"]
scat_y = pop_df["Crime Per Capita"]
plt.scatter(scat_x, scat_y)
plt.xlabel("Median Neighborhood Income (K)")
plt.ylabel("Crime Per Capita")
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(scat_x, scat_y)
regress_values = scat_x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(scat_x,regress_values,"r-")
plt.annotate(line_eq,(100, 1.5),fontsize=15,color="red")

In [ ]:

dates = final_df["Occur Date"]
#create a temporay df to keep the date info
#make sure to import datetime
df = pd.DataFrame({'Occur Date': pd.to_datetime(dates)})
#we need year, month and dayofweek
date_terms = ['year', 'month', 'day','dayofweek']
#generate year, month, day, dayofweek
date_gen = (getattr(df['Occur Date'].dt, i).rename(i) for i in date_terms)
#add the new columns
df = df.join(pd.concat(date_gen, axis=1))

#need to add the info to the crime data
Month = df["month"]
dayofweek = df["dayofweek"]
year = df["year"]

final1_df = final_df.assign(Month = Month,Year = year,Dayofweek = dayofweek)
#final1_df["Dayofweek"].unique()
#Week of the day: 0-Monday, 1-Tuesday.....6-Sunday
months_df = final1_df.groupby(final1_df["Month"])
crimes_per_month = months_df["UCR #"].count()
x_values = np.arange(len(final1_df["Month"].unique()))
tick_locations = [value for value in x_values]
months =["January", "February", "March", "April", "May", "June", "July","August","September",
        "October","November","December"]
plt.figure(figsize=(20,5))
plt.xticks(tick_locations, months, rotation="vertical",fontsize=18)
error = crimes_per_month**(1/2)
plt.bar(x_values, crimes_per_month,yerr=error,align='center', alpha=0.8, ecolor='red')
plt.title("No. of crimes per month",fontsize=24)
plt.ylabel("No of crimes",fontsize=18)
plt.yticks(fontsize=18)

plt.show()
plt.tight_layout()
print("The standard deviation of the distribution = ", crimes_per_month.std())
mean_month = crimes_per_month.mean()
print ("The mean number of crimes per month =",mean_month )

#check if the changes are significant
crimes_month_df = pd.DataFrame([crimes_per_month]).T
crimes_month_df[1]=mean_month
crimes_month_df.columns=["observed crimes", "average crimes"]
#degree of freedom is 11, confidence level=95%
critical_value = stats.chi2.ppf(q = 0.95, df = 11)
print("critical value =",critical_value)
#running the chisq test
print(stats.chisquare(crimes_month_df['observed crimes'], crimes_month_df['average crimes']))
crimes_per_month

In [ ]:
types_of_crimes = final1_df.groupby(final1_df["UCR #"])
x_values= types_of_crimes["UCR #"].unique()
types= types_of_crimes["UCR #"].count()
crime_type_df = pd.DataFrame({"Types of Crimes":types})
#plt.figure(figsize=(20,5))
#plt.bar(x_values, types)

#plt.show()
#plt.tight_layout()
#types_of_crimes["UCR #"].unique())
crime_type_df.plot(kind="bar", figsize=(15,5))

In [ ]:
littypes_per_month = lit_types["UCR Literal"].count()
lit_values = np.arange(len(final1_df["UCR Literal"].unique()))
plt.bar(lit_values, littypes_per_month)

In [ ]:
#types
lit_types =final1_df.groupby(final1_df["UCR Literal"])
lit_type = lit_types["UCR Literal"].count()
lit_type

In [ ]:


summary = pd.DataFrame(final_df.groupby("UCR Literal").size())

# Use DataFrame.plot() in order to create a bar chart of the da
summary.plot(kind="pie", labels=summary.index.values, 
        autopct="%10.1f%%",subplots=True,figsize=(25,25),fontsize=22, startangle=110,legend=False)
#colors = ['lightskyblue', 'red', 'blue', 'green', 'gold']
#plt.pie(df_score_2['Stat'], labels= labels, colors=colors, startangle=90, autopct='%.1f%%')

plt.show()

In [ ]:
final1_df["UCR Literal"].value_counts()
final1_df.loc[final1_df["UCR #"] == 730]

In [ ]:
crimes_dayofweek =final1_df.groupby(final1_df["Dayofweek"])
dayofweek= crimes_dayofweek["UCR #"].count()
x_dayofweek = np.arange(len(final1_df["Dayofweek"].unique()))

tick_locations = [value for value in x_dayofweek]
daysofweek =["Monday", "Tuesday", "Wednesday", "Thursday","Friday", "Saturday", "Sunday"]
plt.figure(figsize=(20,5))
error_week = dayofweek**(1/2)
plt.xticks(tick_locations, daysofweek, rotation="vertical",fontsize=18)
#plt.bar(x_values, crimes_per_month)
plt.bar(x_dayofweek, dayofweek,yerr=error_week,align='center', alpha=0.8, ecolor='red')
plt.title("No. of crimes per day of week",fontsize=24)
plt.ylabel("No of crimes",fontsize=18)
plt.yticks(fontsize=18)
plt.show()
plt.tight_layout()
print("The standard deviation of the distribution = ", dayofweek.std())
mean_week = dayofweek.mean()
print ("The mean number of crimes per month =",mean_week)

#chi square test
crimes_week_df = pd.DataFrame([dayofweek]).T
crimes_week_df[1]=mean_week
crimes_week_df.columns=["observed crimes", "average crimes"]
#degree of freedom is 11, confidence level=95%
critical_value1 = stats.chi2.ppf(q = 0.95, df = 6)
print("critical value =",critical_value1)
#running the chisq test
print(stats.chisquare(crimes_week_df['observed crimes'], crimes_week_df['average crimes']))
dayofweek

In [ ]:

gmaps.configure(g_key)
final_df.head()

In [ ]:
neighborhood_detail_df = final_df.groupby("Neighborhood").mean()
neighborhood_detail_df.reset_index(inplace = True)
neighborhood_detail_df.head()

In [ ]:
neighborhood_locations = neighborhood_detail_df[["Latitude", "Longitude"]]

#hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]

info_box_neighborhood = """
<dl>
<dt>Name</dt><dd>{Neighborhood}</dd>
<dt>Median Income</dt><dd>{Median Income (K)}</dd>
<dt>Population</dt><dd>{Population (K)}</dd>
</dl>
"""
neighborhood_info = [info_box_neighborhood.format(**row) for index, row in neighborhood_detail_df.iterrows()]


In [ ]:
crime_location = final_df[["Latitude", "Longitude"]]

# locations

#neighborhood = final_df["Neighborhood"].astype(float)



fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(crime_location,
                                   dissipating=False, max_intensity=10,
                                   point_radius=.001)
#markers = gmaps.marker_layer(neighborhood_locations, info_box_content = neighborhood_info)
#fig.add_layer(markers)
fig.add_layer(heatmap_layer)


fig


In [ ]:
lit_type

In [ ]:
locations = crimes_df[["Latitude", "Longitude"]]

# locations

rprt_num = crimes_df["Report Number"].astype(float)



fig = gmaps.figure()

crime_layer = gmaps.heatmap_layer(locations,
                                   dissipating=False, max_intensity=7,
                                   point_radius=0.001)

markers = gmaps.marker_layer(neighborhood_locations, info_box_content = neighborhood_info)
fig.add_layer(markers)
fig.add_layer(crime_layer)

fig


In [ ]:
homicide_df = final_df.loc[final_df["UCR Literal"] == 'HOMICIDE']
rob_res_df = final_df.loc[final_df["UCR Literal"] == 'ROBBERY-RESIDENCE']
homicide_locations = homicide_df[["Latitude", "Longitude"]]
rob_res_locations = rob_res_df[["Latitude", "Longitude"]]
auto_df = final_df.loc[final_df["UCR Literal"] == 'AUTO THEFT']
auto_locations = auto_df[["Latitude", "Longitude"]]
assault_df = final_df.loc[final_df["UCR Literal"] == 'AGG ASSAULT']
assault_locations = assault_df[["Latitude", "Longitude"]]

In [ ]:
fig = gmaps.figure()

rob_res_heatmap_layer = gmaps.heatmap_layer(rob_res_locations,
                                   dissipating=False, max_intensity=2,
                                   point_radius=.001)

markers = gmaps.marker_layer(neighborhood_locations, info_box_content = neighborhood_info)
fig.add_layer(markers)
fig.add_layer(rob_res_heatmap_layer)


fig

In [ ]:
fig = gmaps.figure()

auto_heatmap_layer = gmaps.heatmap_layer(auto_locations,
                                   dissipating=False, max_intensity=2,
                                   point_radius=.001)

markers = gmaps.marker_layer(neighborhood_locations, info_box_content = neighborhood_info)
fig.add_layer(markers)
fig.add_layer(auto_heatmap_layer)


fig

In [ ]:
fig = gmaps.figure()

assault_heatmap_layer = gmaps.heatmap_layer(assault_locations,
                                   dissipating=False, max_intensity=5,
                                   point_radius=.001)

markers = gmaps.marker_layer(neighborhood_locations, info_box_content = neighborhood_info)
fig.add_layer(markers)
fig.add_layer(assault_heatmap_layer)


fig

In [ ]:
fig = gmaps.figure()

homicide_heatmap_layer = gmaps.heatmap_layer(homicide_locations,
                                   dissipating=False, max_intensity=2,
                                   point_radius=.001)

markers = gmaps.marker_layer(neighborhood_locations, info_box_content = neighborhood_info)
fig.add_layer(markers)
fig.add_layer(homicide_heatmap_layer)


fig